In [ ]:
# imports
import numpy as np
from obspy import UTCDateTime
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.interpolate import interp1d
from scipy.fftpack import next_fast_len
from utils.model_tools_kurama import logheal_llc
from scipy.special import erf, erfc

# Part A: Model functions describing specific physical influences on seismic velocity

#### Model for earthquake healing
accelerated implementation using low-level callback by Kurama Okubo 

modified to account for better normalization of the healing function by shifting the time of the earthquake to the nearest available time sample

Okubo, K., Delbridge, B. G., & Denolle, M. A. (2024). Monitoring velocity change over 20 years at Parkfield. Journal of Geophysical Research: Solid Earth, 129, e2023JB028084. https://doi.org/10.1029/2023JB028084 

In [ ]:
# b: healing
def func_healing(independent_vars, params, time_quake="2017,09,19,18,14,00"):
    # full implementation of Snieder's healing model from Snieder et al. 2017
    # but faster
    # (c) by Kurama Okubo
    t = independent_vars[0]
    if len(independent_vars) == 2:
        time_quake = independent_vars[1]

    tau_min = 0.1
    tau_max = params[0]
    drop_eq = params[1]
    tquake = UTCDateTime(time_quake).timestamp

    ix_eq_on_timeaxis = np.argmin((t - tquake)**2)
    tquake = t[ix_eq_on_timeaxis]
    
    tax = t - tquake
    ixt = tax >= 0
    tax[~ixt] = 0.0

    dv_quake = np.zeros(len(tax))

    # separate function accelerated by c and low level callback for scipy quad
    dv_quake[ixt] = [logheal_llc(tt, tau_min, tau_max, drop_eq) for tt in tax[ixt]]
    # print("minimum of earthquake DV: ", dv_quake.min())
    dv_quake /= np.log(tau_max/tau_min)
    print("minimum of earthquake DV after normalization: ", dv_quake.min())
    
    # reinterpolate
    #f = interp1d(t_low, dv_quake_low, bounds_error=False)
    return(dv_quake)

# for more than 1 quake in the timeseries
def func_healing_list(independent_vars, params):
    t = independent_vars[0]
    quakes = independent_vars[1]

    dv_quakes = np.zeros(len(t))
    tau_max_list = params[0]
    drop_eq_list = params[1]

    for i in range(len(quakes)):
        dv_quakes += func_healing([t], [tau_max_list[i], drop_eq_list[i]], time_quake=quakes[i])
    return(dv_quakes)

#### Model for pore pressure effect on dvv

In [ ]:
#################################################
# Hydrology: 1-D poroelastic response to rainfall
#################################################
def get_effective_pressure(rhophi, z, rhos):
    p = np.zeros(len(z))
    dz = np.zeros(len(z))
    dz[:-1] = z[1:] - z[:-1]
    dz[-1] = dz[-2]

    p[1: ] += np.cumsum(rhos * 9.81 * dz)[:-1]  # overburden

    # parameter rhophi: rho water * porosity
    p[1: ] -= z[1:] * rhophi[1:] * 9.81 # roughly estimated pore pressure -- just set to hydrostatic pressure here
    return(p)

def model_SW_dsdp(p_in, waterlevel=100.):
    # 1 / vs  del v_s / del p: Derivative of shear wave velocity to effective pressure
    # identical for Walton smooth model and Hertz-Mindlin model
    # input: 
    # p_in (array of int or float): effective pressure (hydrostatic - pore)
    # waterlevel: to avoid 0 division at the free surface. Note that results are sensitive to this parameter.
    # output: 1/vs del vs / del p

    p_in[p_in < waterlevel] = waterlevel
    sens = 1. / (6. * p_in)
    return(sens)

def roeloffs_1depth(t, rain, r, B_skemp, nu, diff,
                    rho, g, waterlevel, model, nfft=None):
    # evaluate Roeloff's response function for a specific depth r
    # input:
    # t: time vector in seconds
    # rain: precipitation time series in m
    # r: depth in m
    # B_skemp: Skempton's coefficient (no unit)
    # nu: Poisson ratio (no unit)
    # diff: Hydraulic diffusivity, m^2/s
    # rho: Density of water (kg / m^3)
    # g: gravitational acceleration (N / kg)
    # waterlevel: to avoid zero division at the surface. Results are not sensitive to the choice of waterlevel
    # model: drained, undrained or both (see Roeloffs, 1988 paper)
    # output: Pore pressure time series at depth r

    # use nfft to try an increase convolution speed
    if nfft is None:
        nfft = len(t)

    dp = rho * g * rain
    dt = t[1] - t[0]  # delta t, sampling (e.g. 1 day)
    diffterm = 4. * diff * np.arange(len(t)) * dt
    diffterm[0] = waterlevel
    diffterm = r / np.sqrt(diffterm)
    
    resp = erf(diffterm)
    rp = np.zeros(nfft)
    rp[0: len(resp)] = resp
    P_ud = np.convolve(rp, dp, "full")[0: len(dp)]
    
    resp = erfc(diffterm)
    rp = np.zeros(nfft)
    rp[0: len(resp)] = resp
    P_d = np.convolve(rp, dp, "full")[0: len(dp)]
    if model == "both":
        P = P_d + B_skemp * (1 + nu) / (3. - 3. * nu) * P_ud
    elif model == "drained":
        P = P_d
    elif model == "undrained":
        P = B_skemp * (1 + nu) / (3. - 3. * nu) * P_ud
    else:
        raise ValueError("Unknown model for Roeloff's poroelastic response. Model must be \"drained\" or \"undrained\" or \"both\".")
    return P

def roeloffs(t, rain, r, B_skemp, nu, diff, rho=1000.0, g=9.81, waterlevel=1.e-12, model="both"):
    s_rain = np.zeros((len(t), len(r)))
    fftN = next_fast_len(len(t))
    for i, depth in enumerate(r):
        p = roeloffs_1depth(t, rain, depth, B_skemp, nu, diff,
                            rho, g, waterlevel, model, nfft=fftN)
        s_rain[:, i] = p
    return(s_rain)


def func_rain(independent_vars, params):
    # This function does the bookkeeping for predicting dv/v from pore pressure change.
    z = independent_vars[0]
    dp_rain = independent_vars[1]
    rhos = independent_vars[2]
    phis = independent_vars[3]
    kernel = independent_vars[4]

    dz = np.zeros(len(z))
    dz[:-1] = z[1:] - z[:-1]
    dz[-1] = dz[-2]

    min_eff_press = params[0]

    rhophi = 1000.0 * phis
    p = get_effective_pressure(rhophi, z, rhos)
    stress_sensitivity = model_SW_dsdp(p, min_eff_press)
    dv_rain = np.dot(-dp_rain, stress_sensitivity * kernel * dz)

    return(dv_rain)

#### Thermoelastic effect

In [ ]:
#################################################
# Thermoelastic effect following Richter et al., 2015
#################################################

def diff_temp_term(t0_surface, t, z, n, diff, w0=2.*np.pi/(365.25*86400.0)):
    gamma = np.sqrt(n * w0 / (2. * diff))
    ts = t0_surface * np.exp(1.j * (n * t * w0 - gamma * z) - gamma * z)
    return(np.real(ts))

def cn(n, t, y, tau=86400.0 * 365.25):
    c = y * np.exp(-1.j * 2 * n * np.pi * t / tau)
    return c.sum()/c.size


def get_temperature_z(t, T_surface, z, thermal_diffusivity,
                      n_fourier_components=6):
    
    T_surface -= T_surface.mean()

    # get Fourier series representation of temperature
    fcoeffs = np.array([cn(n, t - t.min(), T_surface, tau=86400.0 * 365.25) \
        for n in range(n_fourier_components)])

    # get diffusion result
    difftemp = np.zeros((len(t), len(z)))
    for ix, zz in enumerate(z):
        for n, fc in enumerate(fcoeffs):
            difftemp[:, ix] += np.array([diff_temp_term(fc, tt, zz, n, thermal_diffusivity) \
            for tt in t - t.min()])

    # return diffusion result
    return(difftemp)

def func_temp(independent_vars, params):

    t = independent_vars[0]
    z = independent_vars[1]
    kernel = independent_vars[2]
    dp_temp = independent_vars[3]

    dz = np.zeros(len(z))
    dz[:-1] = z[1:] - z[:-1]
    dz[-1] = dz[-1]

    assert dz[0] > 0.0
    sensitivity_factor = params[0]
    dv_temp = sensitivity_factor * np.dot(dp_temp, kernel * dz)
    return(dv_temp)

#### Linear trend (in Mexico City probably subsidence-related)

In [ ]:
#################################################
# Linear velocity increase / decrease
#################################################
def func_lin(independent_vars, params):
    # linear trend
    t = independent_vars[0]
    slope = params[0]
    const = params[1]

    dv_y = slope * (t - t.min()) + const
    return(dv_y)


#### Here is a space for the function for lake level dependence:

##### additional convenience functions

In [ ]:
# Function to replace NaN values with nearest non-NaN values
# by ChatGPT
# prompt: I have a numpy array in Python that contains several NaN-values, and I would like to replace them by the nearest finite value.
def replace_nan_with_nearest(arr, interpolation_method="nearest"):
    indices = np.arange(len(arr))
    valid = ~np.isnan(arr)
    
    # Nearest-neighbor interpolation
    f = interp1d(indices[valid], arr[valid], bounds_error=False,
                 fill_value="extrapolate", kind=interpolation_method)
    
    arr[~valid] = f(indices[~valid])
    return arr


# Part B: Input data


#### Load vs sensitivity kernel for Rayleigh waves

In [ ]:
# Load sensitivity kernel
data_dc_ds = np.loadtxt('cdmx_example_data/kernels_psv.unm.f=0.500.c=1117.595', skiprows=3)

depth_in_meters = np.abs(6371000. - data_dc_ds[:, 0])
K_vs = data_dc_ds[:, -2] + data_dc_ds[:, -3]

depth_in_meters = depth_in_meters[::-1]
K_vs = K_vs[::-1]

linestyle_strs = ['solid', 'dashed', 'dotted', 'dashdot', 'solid']
plt.plot(K_vs, depth_in_meters, linewidth=2) #

plt.xlabel('Sensitivity (dC/dVs)', fontsize=12)
plt.ylabel('Depth')

plt.ylim([3000., 0.])

plt.grid()
plt.show()

#### Load density

In [ ]:
# load the density profile
rho = np.ones(len(depth_in_meters)) * 1900.



#### Define the meteoreological data: timestamp, temperature, rain

In [ ]:
# define a function to get meteo data
# needed: Timestamp,  temperature, rain
def get_met_data(plot):
    timestamps = np.load("cdmx_example_data/timestamps_unm_BHZ_BHN_fmin0.5Hz.npy")
    rain = np.load("cdmx_example_data/rain_unm_BHZ_BHN_fmin0.5Hz.npy")
    temperature = np.load("cdmx_example_data/temp_unm_BHZ_BHN_fmin0.5Hz.npy")

    if plot:
        plt.plot(timestamps, rain * 1000, "b")
        plt.ylabel("Rain / mm")
        plt.xticks(rotation=45)
        xt = plt.xticks()
        plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
        plt.grid()
        plt.show()
    
        plt.plot(timestamps, temperature, "r")
        plt.grid()
        plt.xticks(rotation=45)
        xt = plt.xticks()
        plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
        plt.ylabel("Temperature / degree C")
        plt.show()
        
    
    
    df_out = pd.DataFrame(columns=["dates", "timestamps", "rain", "Temp_C"])
    df_out["timestamps"] = timestamps
    
    df_out["rain"] = rain
    df_out["Temp_C"] = temperature
    
    #plt.plot(df_interp.timestamps.values[1:], np.diff(df_interp.timestamps.values))
    return(df_out)

df_meteo = get_met_data(plot=True)

#### Load dvv data
needed: timestamps, dvv, correlation coefficient (error is actually only needed for inversion)

In [ ]:
# load dvv data. 
dvv = np.load("cdmx_example_data/data_unm_BHZ_BHN_fmin0.5Hz.npy")[0]

# Here, the dvv data have the same time stamp as the meteo data.
# if this is not the case, then load also the timestamps
tstamps = df_meteo.timestamps.values

# to run the model faster, either downsample the data
# or cut off years, or both
# meteo data are missing prior to about 2002
tstamps_low = np.linspace(UTCDateTime("2014,150").timestamp, tstamps.max(), 19 * 24)
f = interp1d(tstamps, dvv, kind="cubic")
dvv_low = f(tstamps_low)


# replace nans by a finite value!! Here this step has been done earlier
dvv_qc = dvv - dvv[0]
dvv_qc_low = dvv_low - dvv_low[0]

plt.plot(df_meteo["timestamps"], dvv_qc * 100, "k")
plt.plot(tstamps_low, dvv_qc_low * 100, "orange")
plt.grid()
plt.xticks(rotation=45)
xt = plt.xticks()
plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
plt.ylabel("dv/v / %")
plt.show()

tstamps = tstamps_low
dvv_qc = dvv_qc_low

# Part C: Modeling the dv/v time series

#### Model parameters: adapt as needed

In [ ]:
# general inputs
# ==============
# which effects to consider in the model?
consider_effects = ["linear", "quake", "rain", ] #["quake", "temperature", "linear", "rain"]


# earthquake-related parameters
# =============================
# origin time of the earthquake
qtimes = [UTCDateTime("2017-09-19T18:14:40"), UTCDateTime("2020-06-23T15:29:05")]

# starting values
# list of the log10 of maximum relaxation times (one per earthquake)
log10_tau_maxs = [np.log10(86400. * 365 * 10),np.log10(86400. * 365 * 0.5),]
# list of the velocity drops (one per earthquake).
# currently, there is an issue with normalization
# so units currently unknown -- tbd!!
drops = [0.1, 0.002]


# rain-related parameters
# =======================
# undrained Poisson ratio of the material
nu_undrained = 0.4
# hydraulic diffusivity
diff_in = 5.e-4
# parts of Roeloff's model to consider: undrained term, drained term or both
# the undrained term is the pore pressure change due to diffusion, the drained term the elastic load of the rain
# if you choose "both", they are added
roeloffs_terms = "both"  # "drained", "undrained" or "both"
B = 1  # Skempton's B
# log10 of the minimum effective pressure. This parameter is the effective pressure at the surface / at the level of the observation.
# hydrostatic pressure - pore pressure
# in N
# starting value
log10_p0 = np.log10(3)

# temperature-related parameters
# ===============================
# thermal diffusivity of the soil
diff_in_temp = 1.e-6
# number of Fourier terms to describe the seasonal temperature variation
n_fourier_terms_temperature = 2
# log10 of the sensitivity to thermoelastic changes
log10_tsens = -3

# linear increase parameters
# ==========================
slope = 0.0018 / (86400. * 365.) # in rate per year, starting value
offset = 0  # offset, starting value

In [ ]:
# running the model

# preparing meteo-dependent models
f = interp1d(df_meteo.timestamps.values, df_meteo.rain.values, bounds_error=False, fill_value="extrapolate")
rain_m = f(tstamps)
rain_m -= np.mean(rain_m)
f = interp1d(df_meteo.timestamps.values, df_meteo.Temp_C.values, bounds_error=False, fill_value="extrapolate")
temperature_C = f(tstamps)

dp_rain = roeloffs(tstamps, rain_m, depth_in_meters, B, nu_undrained, diff_in, model=roeloffs_terms)

# for temperature we need a finer depth sampling, as it affects mostly the upper meters
# for the chosen model
depth_in_meters_T = np.concatenate([np.arange(0, depth_in_meters[2] + 0.5, 0.5), depth_in_meters[3:]])
dp_temp = get_temperature_z(tstamps, temperature_C, depth_in_meters_T,
    diff_in_temp, n_fourier_components=n_fourier_terms_temperature)



# uncomment below to plot some rough sketches of the pore pressure effect

xv, yv = np.meshgrid(tstamps, -depth_in_meters, indexing="ij")
plt.pcolormesh(xv, yv, dp_rain)
plt.xticks(rotation=45)
xt = plt.xticks()
plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
plt.ylim(-500, 0)
plt.show()

plt.plot(tstamps, rain_m / rain_m.max())
plt.plot(tstamps, dp_rain[:, 0] / dp_rain.max())
plt.legend(["rain normalized", "pore pressure normalized"])
plt.show()

# uncomment below to plot some rough sketches of the thermoelastic effect
xv, yv = np.meshgrid(tstamps, -depth_in_meters_T, indexing="ij")
print(dp_temp.max(), dp_temp.min())
plt.pcolormesh(xv, yv, dp_temp)
plt.colorbar()
plt.ylim(-30, 0)
print(np.abs(dp_temp).min())


# interpolate the kernel to the same depths as the temperature data
f = interp1d(depth_in_meters, K_vs, bounds_error=False, fill_value="extrapolate", kind="nearest")
K_vs_T = f(depth_in_meters_T)

# guess a porosity
phi = np.ones(len(rho)) * 0.15

In [ ]:
# Calling model "components"

# independent variables in this model: time, depth, surface wave sensitivity kernel (depths equal to depth array)
# surface wave sensitivity kernel for temperature effect (depths equal to temperature depth array refined at top)
# density, porosity, rain in m, temperature in degrees and earthquake integer timestamps of origin time

# to model precipitation effect:
independent_variables_rain = [depth_in_meters, dp_rain, rho, phi, K_vs]
parameters_rain = [10. ** log10_p0]
effect_rain = func_rain(independent_variables_rain, parameters_rain)

# to model temperature effect
independent_variables_temp = [tstamps, depth_in_meters_T, K_vs_T, dp_temp]
parameters_temp = [10. ** log10_tsens]
effect_temp = func_temp(independent_variables_temp, parameters_temp)

# to model linear increase (trend)
independent_variables_lin = [tstamps]
parameters_lin = [slope, offset]
effect_lin = func_lin(independent_variables_lin, parameters_lin)

# to model earthquake velocity drops and their recovery (Snieder 2017)
independent_variables_quake = [tstamps, qtimes]
parameters_quake = [[10. ** ltm for ltm in log10_tau_maxs], drops]
effect_quake = func_healing_list(independent_variables_quake, parameters_quake)

# Here is space for evaluating the lake level function....

### The following cell plots the data and model results for 1 source and saves results to a csv

In [ ]:
model = np.zeros(len(tstamps))
if "rain" in consider_effects:
    model += effect_rain * 100.  # in % 
if "temperature" in consider_effects:
    model += effect_temp * 100.  # in %
if "quake" in consider_effects:
    model+= effect_quake * 100.  # in %
if "linear" in consider_effects:
    model+= effect_lin * 100.  # in %

plt.figure(figsize=(12, 4))
plt.plot(tstamps, model , linewidth=1.5, label=f"model {[p for p in consider_effects]}")
plt.plot(tstamps, dvv_qc * 100, "k", linewidth=1.5, label="observed dv/v")

# you can also plot the single terms
#plt.plot(tstamps, effect_rain * 100,  alpha=0.5, label="precipitation effect")
#plt.plot(tstamps, effect_temp * 100, alpha=0.5, label="thermoelastic effect")
#plt.plot(tstamps, effect_quake * 100, alpha=0.5, label="earthquake drop & healing")

plt.xticks(rotation=45)
xt = plt.xticks()
plt.xticks(xt[0][1:-1], [UTCDateTime(xtt).strftime('%Y-%m') for xtt in xt[0]][1:-1])
plt.ylabel("dv/v / %")
plt.ylim(-2, 1)
plt.legend()

df_out = pd.DataFrame(columns=["timestamps", "dvv_observed", "dvv_precipitation", "dvv_temperature", "dvv_earthquakes"])
df_out["timestamps"] = tstamps
df_out["dvv_observed"] = dvv_qc
df_out["dvv_precipitation"] = effect_rain
df_out["dvv_temperature"] = effect_temp
df_out["dvv_earthquakes"] = effect_quake
df_out["dvv_linear"] = effect_lin

df_out.to_csv(f"result.csv")